Ant colony optimization for solving the Traveling salesman problem

In [1]:
import numpy as np


In [2]:
class AntColony:
    def __init__(self, distances, n_ants, n_iterations, decay, alpha=1, beta=3):
        self.distances = distances
        self.pheromone = np.ones_like(distances, dtype=float)
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        n_cities = len(self.distances)
        best_route = None
        best_distance = np.inf

        for _ in range(self.n_iterations):
            routes = self.generate_routes()
            self.update_pheromone(routes)

            shortest_route_index = np.argmin([self.route_distance(route) for route in routes])
            shortest_distance = self.route_distance(routes[shortest_route_index])

            if shortest_distance < best_distance:
                best_distance = shortest_distance
                best_route = routes[shortest_route_index]

            self.pheromone *= self.decay

        return best_route, best_distance

    def generate_routes(self):
        n_cities = len(self.distances)
        routes = []

        for _ in range(self.n_ants):
            visited = [0]
            unvisited = list(range(1, n_cities))

            while unvisited:
                next_city = self.select_next_city(visited[-1], unvisited)
                visited.append(next_city)
                unvisited.remove(next_city)

            route = visited + [visited[0]]  # Return to start city
            routes.append(route)

        return routes

    def select_next_city(self, current_city, unvisited):
        pheromone = self.pheromone[current_city, unvisited]
        heuristic = 1 / self.distances[current_city, unvisited]
        probabilities = (pheromone ** self.alpha) * (heuristic ** self.beta)
        probabilities /= probabilities.sum()
        next_city = np.random.choice(unvisited, p=probabilities)
        return next_city

    def route_distance(self, route):
        distance = 0
        for i in range(len(route) - 1):
            distance += self.distances[route[i], route[i + 1]]
        return distance

    def update_pheromone(self, routes):
        for route in routes:
            distance = self.route_distance(route)
            for i in range(len(route) - 1):
                self.pheromone[route[i], route[i + 1]] += 1 / distance
                self.pheromone[route[i + 1], route[i]] += 1 / distance
                

In [3]:
# Define distances between cities (replace this with your actual data)
distances = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

# Set parameters
n_ants = 10
n_iterations = 100
decay = 0.1

# Create and run AntColony
aco = AntColony(distances, n_ants, n_iterations, decay)
best_route, best_distance = aco.run()

print("Best Route:", best_route)
print("Best Distance:", best_distance)


Best Route: [0, 1, 3, 2, 0]
Best Distance: 80
